In [0]:
from pyspark.sql.functions import *

In [0]:
start_date = "2024-01-01"
end_date = "2025-12-01"

In [0]:
df = spark.createDataFrame([(f'{start_date}', f'{end_date}')], ["start", "end"])


In [0]:
df.display()

In [0]:
df = df.withColumn('date_sequence', sequence(to_date(col('start')), to_date(col('end')), expr("interval 1 month")))
df = df.withColumn('date_sequence', explode(col('date_sequence'))).select("date_sequence")
df.display()


In [0]:
df = df.withColumn('date_key', date_format('date_sequence', 'yyyyMM').cast('int'))\
    .withColumn('year', year('date_sequence'))\
    .withColumn('month_name', date_format('date_sequence', 'MMMM'))\ 
    .withColumn('month_short_name', monthname('date_sequence'))\
    .withColumn('quarter', concat(lit('Q'), quarter('date_sequence')))\
    .withColumn('year_quarter', concat('year', lit('-'), 'quarter'))

df = df.withColumnRenamed("date_sequence", "month_start_date")

#.withColumn('month_short_name', monthname('date_sequence'))\. This is can also be written as date_format('date_sequence', 'MMM')
df.display()


In [0]:
df.display()

In [0]:
df.write.mode('overwrite').format('delta').saveAsTable('sportats.gold.dim_date')